In [1]:
# #Import all tables from mysql database into hdfs as avro data files. use compression and the compression codec should be snappy. data warehouse directory should be retail_stage.db
# '''sqoop import-all-tables -Dmapreduce.job.user.classpath.first=true --connect jdbc:mysql://ms.itversity.com/retail_db
# --username retail_user --password itversity  --compression-codec snappy 
# --as-avrodatafile -m 1 --warehouse-dir /user/indirameduri/avrodata'''


**Write query in hive that shows all orders belonging to a certain day. This day is when the most orders were placed. select data from orders_sqoop.**

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:2.4.5 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Arun3').master('yarn').getOrCreate()

In [5]:
orders = spark.read.csv('/public/retail_db/orders',inferSchema=True).toDF('order_id','order_date','order_cust_id','order_status')

Py4JJavaError: An error occurred while calling o123.csv.
: java.util.ServiceConfigurationError: org.apache.spark.sql.sources.DataSourceRegister: Provider org.apache.spark.sql.avro.AvroFileFormat could not be instantiated
	at java.util.ServiceLoader.fail(ServiceLoader.java:232)
	at java.util.ServiceLoader.access$100(ServiceLoader.java:185)
	at java.util.ServiceLoader$LazyIterator.nextService(ServiceLoader.java:384)
	at java.util.ServiceLoader$LazyIterator.next(ServiceLoader.java:404)
	at java.util.ServiceLoader$1.next(ServiceLoader.java:480)
	at scala.collection.convert.Wrappers$JIteratorWrapper.next(Wrappers.scala:43)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:54)
	at scala.collection.TraversableLike$class.filterImpl(TraversableLike.scala:247)
	at scala.collection.Trave

In [4]:
orders.show()

+--------+-------------------+-------------+---------------+
|order_id|         order_date|order_cust_id|   order_status|
+--------+-------------------+-------------+---------------+
|       1|2013-07-25 00:00:00|        11599|         CLOSED|
|       2|2013-07-25 00:00:00|          256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|        12111|       COMPLETE|
|       4|2013-07-25 00:00:00|         8827|         CLOSED|
|       5|2013-07-25 00:00:00|        11318|       COMPLETE|
|       6|2013-07-25 00:00:00|         7130|       COMPLETE|
|       7|2013-07-25 00:00:00|         4530|       COMPLETE|
|       8|2013-07-25 00:00:00|         2911|     PROCESSING|
|       9|2013-07-25 00:00:00|         5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|         5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|          918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|         1837|         CLOSED|
|      13|2013-07-25 00:00:00|         9149|PENDING_PAYMENT|
|      14|2013-07-25 00:

In [5]:
orderItems = spark.read.csv('/public/retail_db/order_items',inferSchema=True).toDF('oi_id','oi_order_id','oi_product_id','oi_quantity','oi_subtotal','oi_product_price')

In [6]:
orderItems.show()

+-----+-----------+-------------+-----------+-----------+----------------+
|oi_id|oi_order_id|oi_product_id|oi_quantity|oi_subtotal|oi_product_price|
+-----+-----------+-------------+-----------+-----------+----------------+
|    1|          1|          957|          1|     299.98|          299.98|
|    2|          2|         1073|          1|     199.99|          199.99|
|    3|          2|          502|          5|      250.0|            50.0|
|    4|          2|          403|          1|     129.99|          129.99|
|    5|          4|          897|          2|      49.98|           24.99|
|    6|          4|          365|          5|     299.95|           59.99|
|    7|          4|          502|          3|      150.0|            50.0|
|    8|          4|         1014|          4|     199.92|           49.98|
|    9|          5|          957|          1|     299.98|          299.98|
|   10|          5|          365|          5|     299.95|           59.99|
|   11|          5|      

In [7]:
joinData = orders.join(orderItems,orders.order_id== orderItems.oi_order_id)
joinData.show()

+--------+-------------------+-------------+---------------+-----+-----------+-------------+-----------+-----------+----------------+
|order_id|         order_date|order_cust_id|   order_status|oi_id|oi_order_id|oi_product_id|oi_quantity|oi_subtotal|oi_product_price|
+--------+-------------------+-------------+---------------+-----+-----------+-------------+-----------+-----------+----------------+
|       1|2013-07-25 00:00:00|        11599|         CLOSED|    1|          1|          957|          1|     299.98|          299.98|
|       2|2013-07-25 00:00:00|          256|PENDING_PAYMENT|    2|          2|         1073|          1|     199.99|          199.99|
|       2|2013-07-25 00:00:00|          256|PENDING_PAYMENT|    3|          2|          502|          5|      250.0|            50.0|
|       2|2013-07-25 00:00:00|          256|PENDING_PAYMENT|    4|          2|          403|          1|     129.99|          129.99|
|       4|2013-07-25 00:00:00|         8827|         CLOSED|  

In [8]:
mostorders = joinData.groupBy('order_date').count().sort('count',ascending=False).limit(10)
mostorders.show()

+-------------------+-----+
|         order_date|count|
+-------------------+-----+
|2013-11-03 00:00:00|  860|
|2014-02-01 00:00:00|  757|
|2013-11-14 00:00:00|  743|
|2013-12-26 00:00:00|  734|
|2013-09-25 00:00:00|  730|
|2013-10-13 00:00:00|  727|
|2013-11-24 00:00:00|  726|
|2014-02-19 00:00:00|  713|
|2014-07-20 00:00:00|  712|
|2013-10-04 00:00:00|  709|
+-------------------+-----+



In [9]:
from pyspark.sql.functions import date_format
month = orders.select(date_format('order_date','yyyy-MM')).orderBy(date_format('order_date','MM')).dropDuplicates()
month.show()

+--------------------------------+
|date_format(order_date, yyyy-MM)|
+--------------------------------+
|                         2013-09|
|                         2013-12|
|                         2014-01|
|                         2014-03|
|                         2013-11|
|                         2014-02|
|                         2013-07|
|                         2014-05|
|                         2013-08|
|                         2014-04|
|                         2014-06|
|                         2013-10|
|                         2014-07|
+--------------------------------+



In [10]:
spark.sql('use indirameduri')

DataFrame[]

In [11]:
spark.sql('show tables').show()

+------------+-----------+-----------+
|    database|  tableName|isTemporary|
+------------+-----------+-----------+
|indirameduri|       arun|      false|
|indirameduri|departments|      false|
|indirameduri|       drop|      false|
|indirameduri|order_items|      false|
|indirameduri|     orders|      false|
|indirameduri|    parq_t1|      false|
|indirameduri|products_im|      false|
|indirameduri|temp_orders|      false|
+------------+-----------+-----------+



In [12]:
spark.sql('create table ordersAvro(order_id int,order_date date,order_cust_id int,order_status string) row format delimited fields terminated by "," stored as avro partitioned by date_format("order_date","yyyy-MM")')

ParseException: '\nextraneous input \'date_format\' expecting \'(\'(line 1, pos 168)\n\n== SQL ==\ncreate table ordersAvro(order_id int,order_date date,order_cust_id int,order_status string) row format delimited fields terminated by "," stored as avro partitioned by date_format("order_date","yyyy-MM")\n------------------------------------------------------------------------------------------------------------------------------------------------------------------------^^^\n'